In [3]:
import os
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization, Input, GlobalAveragePooling2D
from tensorflow.keras.losses import CategoricalCrossentropy
import random
from sklearn.model_selection import train_test_split
import shutil

In [2]:
## 1.0. Rebalanceamento
def undersample(src_dir, target_dir, target_count):
    src_len = len(os.listdir(src_dir))
    
    if not os.path.exists(target_dir):
        target_dir = os.path.join("imagens", target_dir)
        os.makedirs(target_dir)
    
    images = os.listdir(src_dir)
    
    selected_images = random.sample(images, target_count)
    
    for img in images:
        if img in selected_images:
            img_src_path = os.path.join(src_dir, img)
            img_target_path = os.path.join(target_dir, img)
            shutil.copy(img_src_path, img_target_path)
            print(f'Imagem movida: {img}')
    
    print(f'Undersampling concluído para a pasta {src_dir}.')

dataset_dir = os.path.join(os.getcwd(), 'imagens')
dataset_morango_dir = os.path.join(dataset_dir, 'morango')
dataset_pessego_dir = os.path.join(dataset_dir, 'pessego')
dataset_roma_dir = os.path.join(dataset_dir, 'roma')

# Define o número alvo de imagens para cada categoria (o menor número de imagens entre todas as categorias)
target_count = min(len(os.listdir(dataset_morango_dir)), len(os.listdir(dataset_pessego_dir)), len(os.listdir(dataset_roma_dir)))

# Aplicar undersampling a cada categoria
undersample(dataset_morango_dir, "morango_rebalanceado", target_count)
undersample(dataset_pessego_dir, "pessego_rebalanceado", target_count)
undersample(dataset_roma_dir, "roma_rebalanceado", target_count)

# 1.1. Dataset das imagens
dataset_dir = os.path.join(os.getcwd(), 'imagens')
dataset_morango_dir = os.path.join(dataset_dir, 'morango_rebalanceado')
dataset_pessego_dir = os.path.join(dataset_dir, 'pessego_rebalanceado')
dataset_roma_dir = os.path.join(dataset_dir, 'roma_rebalanceado')

dataset_morango_len = len(os.listdir(dataset_morango_dir))
dataset_pessego_len = len(os.listdir(dataset_pessego_dir))
dataset_roma_len = len(os.listdir(dataset_roma_dir))

print(f'Contagem de imagens de morango rebalanceado: {dataset_morango_len}')
print(f'Contagem de imagens de pêssego rebalanceado: {dataset_pessego_len}')
print(f'Contagem de imagens de romã rebalanceado: {dataset_roma_len}')


Imagem movida: ms_1.jpg
Imagem movida: ms_10.jpg
Imagem movida: ms_100.jpg
Imagem movida: ms_101.jpg
Imagem movida: ms_102.jpg
Imagem movida: ms_103.jpg
Imagem movida: ms_104.jpg
Imagem movida: ms_105.jpg
Imagem movida: ms_106.jpg
Imagem movida: ms_107.jpg
Imagem movida: ms_108.jpg
Imagem movida: ms_109.jpg
Imagem movida: ms_11.jpg
Imagem movida: ms_110.jpg
Imagem movida: ms_111.jpg
Imagem movida: ms_112.jpg
Imagem movida: ms_113.jpg
Imagem movida: ms_114.jpg
Imagem movida: ms_115.jpg
Imagem movida: ms_116.jpg
Imagem movida: ms_117.jpg
Imagem movida: ms_118.jpg
Imagem movida: ms_119.jpg
Imagem movida: ms_12.jpg
Imagem movida: ms_120.jpg
Imagem movida: ms_121.jpg
Imagem movida: ms_122.jpg
Imagem movida: ms_123.jpg
Imagem movida: ms_124.jpg
Imagem movida: ms_125.jpg
Imagem movida: ms_126.jpg
Imagem movida: ms_127.jpg
Imagem movida: ms_128.jpg
Imagem movida: ms_129.jpg
Imagem movida: ms_13.jpg
Imagem movida: ms_130.jpg
Imagem movida: ms_131.jpg
Imagem movida: ms_132.jpg
Imagem movida: ms_

In [9]:
## 2. Definição e separação dos dados de treinamento e dados de teste
projeto_dir = os.getcwd() 
dataset_treinamento_dir = os.path.join(projeto_dir, 'imagens_treinamento') 
dataset_teste_dir = os.path.join(projeto_dir, 'imagens_teste') 

proporcao_dataset_treinamento = 0.7 # 70% será utilizado no treinamento
proporcao_dataset_teste = 1 - proporcao_dataset_treinamento # 30% será utilizado no teste
random_seed = 42 

classes = ['morango_rebalanceado', 'pessego_rebalanceado', 'roma_rebalanceado']

def split_dataset_to_train_and_test(classe):
    dataset_treinamento_classe_dir = os.path.join(dataset_treinamento_dir, classe) 
    dataset_teste_classe_dir = os.path.join(dataset_teste_dir, classe) 

    dataset_classe_dir = os.path.join(projeto_dir, 'imagens', classe)
    imagens_classe = [os.path.join(dataset_classe_dir, img) for img in os.listdir(dataset_classe_dir)]

    imagens_treinamento_classe, imagens_teste_classe = train_test_split(imagens_classe, test_size=proporcao_dataset_teste, random_state=random_seed) 

    os.makedirs(dataset_treinamento_classe_dir, exist_ok=True)
    os.makedirs(dataset_teste_classe_dir, exist_ok=True)
    
    for imagem in imagens_treinamento_classe:
        shutil.copy(imagem, dataset_treinamento_classe_dir) 

    for imagem in imagens_teste_classe:
        shutil.copy(imagem, dataset_teste_classe_dir) 

# Verifica se os diretórios de treinamento e teste já existem
if not os.path.exists(dataset_treinamento_dir) or not os.path.exists(dataset_teste_dir):
    # Cria os diretórios de treinamento e teste
    os.makedirs(dataset_treinamento_dir, exist_ok=True)
    os.makedirs(dataset_teste_dir, exist_ok=True)

    for classe in classes:
        split_dataset_to_train_and_test(classe)

In [10]:
## 3. Como ficou dataset de treinamento e de testes:

dataset_treinamento_morango_len = len(os.listdir(os.path.join(dataset_treinamento_dir, 'morango_rebalanceado')))
dataset_teste_morango_len = len(os.listdir(os.path.join(dataset_teste_dir, 'morango_rebalanceado')))
dataset_treinamento_pessego_len = len(os.listdir(os.path.join(dataset_treinamento_dir, 'pessego_rebalanceado')))
dataset_teste_pessego_len = len(os.listdir(os.path.join(dataset_teste_dir, 'pessego_rebalanceado')))
dataset_treinamento_roma_len = len(os.listdir(os.path.join(dataset_treinamento_dir, 'roma_rebalanceado')))
dataset_teste_roma_len = len(os.listdir(os.path.join(dataset_teste_dir, 'roma_rebalanceado')))

print(f'Contagem de imagens de morango para treinamento: {dataset_treinamento_morango_len}')
print(f'Contagem de imagens de morango para teste: {dataset_teste_morango_len}')
print(f'Contagem de imagens de pêssego para treinamento: {dataset_treinamento_pessego_len}')
print(f'Contagem de imagens de pêssego para teste: {dataset_teste_pessego_len}')
print(f'Contagem de imagens de romã para treinamento: {dataset_treinamento_roma_len}')
print(f'Contagem de imagens de romã para teste: {dataset_teste_roma_len}')


Contagem de imagens de morango para treinamento: 174
Contagem de imagens de morango para teste: 76
Contagem de imagens de pêssego para treinamento: 174
Contagem de imagens de pêssego para teste: 76
Contagem de imagens de romã para treinamento: 174
Contagem de imagens de romã para teste: 76


In [11]:
## 4. Pré-processamento das imagens
import os
import numpy as np
import tensorflow as tf

input_shape = (300, 300, 3)

def carregar_dados_do_diretorio(directory, label_mode='categorical', batch_size=32):
    data = []
    labels = []
    class_names = sorted(os.listdir(directory))
    
    class_to_index = {class_name: i for i, class_name in enumerate(class_names)}
    
    for class_name in class_names:
        class_dir = os.path.join(directory, class_name)
        for image_file in os.listdir(class_dir):
            image_path = os.path.join(class_dir, image_file)
            image = tf.keras.preprocessing.image.load_img(image_path, target_size=(300, 300))
            image_array = tf.keras.preprocessing.image.img_to_array(image)
            data.append(image_array)
            labels.append(class_to_index[class_name])
    
    if label_mode == 'categorical':
        labels = tf.keras.utils.to_categorical(labels)
    
    dataset = tf.data.Dataset.from_tensor_slices((data, labels))
    dataset = dataset.batch(batch_size)
    
    return dataset

data_set_treinamento = carregar_dados_do_diretorio(dataset_treinamento_dir)
data_set_teste = carregar_dados_do_diretorio(dataset_teste_dir)

In [6]:
## 5. Definição da arquitetura da rede neural do modelo
import tensorflow as tf

base_model = tf.keras.applications.ResNet50(weights='imagenet', include_top=False, input_shape=input_shape)

# Congela as camadas do modelo base para que não sejam treinadas novamente
base_model.trainable = False

# Adiciona camadas personalizadas ao final da ResNet50
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Input(shape=(300,300,3)))
model.add(base_model)
model.add(tf.keras.layers.GlobalAveragePooling2D())
model.add(tf.keras.layers.Dense(256, activation='relu'))
model.add(tf.keras.layers.Dropout(0.5))
model.add(tf.keras.layers.Dense(3, activation='softmax')) 

model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model.summary()




Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet50 (Functional)       (None, 10, 10, 2048)      23587712  
                                                                 
 global_average_pooling2d (  (None, 2048)              0         
 GlobalAveragePooling2D)                                         
                                                                 
 dense (Dense)               (None, 256)               524544    
                                                                 
 dropout (Dropout)           (None, 256)               0         
                                                                 
 dense_1 (Dense)             (None, 3)                 771       
                                                                 
Total params: 24113027 (91.98 MB)
Trainable params: 525315 (2.00 MB)
Non-trainable params: 23587712 (89.98 MB)
________

In [13]:
## 6. Treinamento do Modelo

# Treina apenas as camadas adicionadas (as últimas camadas) do modelo
history = model.fit(
    data_set_treinamento, 
    epochs=10,  
    batch_size=32  
)

loss, accuracy = model.evaluate(data_set_teste)

model_dir = 'modelo'
os.makedirs(model_dir, exist_ok=True)
model_path = os.path.join(model_dir, 'modelo2.keras')
model.save(model_path)

print("Modelo treinado salvo em:", model_path)

8/8 [==============================] - 13s 2s/step - loss: 0.0434 - accuracy: 0.9912
Modelo treinado salvo em: modelo\modelo2.keras


In [34]:
## 7. Avaliação com F1 Score

import os
import numpy as np
import tensorflow as tf
from sklearn.metrics import f1_score

input_shape = (300, 300, 3)

# Função para pré-processar
def preprocess_dataset(directory, target_size=(300, 300), label_mode='categorical', batch_size=32):
    data = []
    labels = []
    class_names = sorted(os.listdir(directory))

    class_to_index = {class_name: i for i, class_name in enumerate(class_names)}
    
    for class_name in class_names:
        class_dir = os.path.join(directory, class_name)
        for image_file in os.listdir(class_dir):
            image_path = os.path.join(class_dir, image_file)
            image = tf.keras.preprocessing.image.load_img(image_path, target_size=target_size)
            image_array = tf.keras.preprocessing.image.img_to_array(image)
            data.append(image_array)
            labels.append(class_to_index[class_name])
    
    if label_mode == 'categorical':
        labels = tf.keras.utils.to_categorical(labels)
    
    dataset = tf.data.Dataset.from_tensor_slices((data, labels))
    dataset = dataset.batch(batch_size)
    
    return dataset

dataset_treinamento_dir = os.path.join(os.getcwd(), 'imagens_treinamento')
dataset_teste_dir = os.path.join(os.getcwd(), 'imagens_teste')

data_set_treinamento = preprocess_dataset(dataset_treinamento_dir)
data_set_teste = preprocess_dataset(dataset_teste_dir)

model_path = os.path.join(os.getcwd(), 'modelo', 'modelo2.keras')
model = tf.keras.models.load_model(model_path)

y_pred = model.predict(data_set_teste)
y_pred_classes = np.argmax(y_pred, axis=1)

y_true = []
for images, labels in data_set_teste:
    y_true.extend(np.argmax(labels, axis=1))

f1 = f1_score(y_true, y_pred_classes, average='weighted')

print("F1 Score:", f1)


8/8 [==============================] - 18s 2s/step
F1 Score: 0.9912280701754386


In [3]:
## Teste

import tensorflow as tf
import numpy as np
import os

def preprocess_image(image_path, target_size=(300, 300)):
    image = tf.keras.preprocessing.image.load_img(image_path, target_size=target_size)
    image_array = tf.keras.preprocessing.image.img_to_array(image)
    image_array = np.expand_dims(image_array, axis=0)
    return image_array

dataset_treinamento_dir = os.getcwd() + '/imagens_treinamento'

class_names = sorted(os.listdir(dataset_treinamento_dir))

image_path = os.getcwd() + '/imagens_teste/roma_rebalanceado/rs_18.jpg'

processed_image = preprocess_image(image_path)

model = tf.keras.models.load_model(os.getcwd() + '/modelo/modelo2.keras')

predictions = model.predict(processed_image)

predicted_class_index = np.argmax(predictions, axis=1)

# Mapeia o índice da classe de volta para o nome da classe
predicted_class_name = class_names[predicted_class_index[0]]

print(f'A imagem pertence à classe: {predicted_class_name}')


1/1 [==============================] - 1s 1s/step
A imagem pertence à classe: roma_rebalanceado
